# 한글 데이터셋 모델 코드 (케라스 모델로 훈련하는)

In [ ]:
# ## 2-1. Keras 모델 생성 및 훈련
# ## 2-1. Keras model initialization and training

# import keras
# import keras.utils as utils

# from keras import backend as K
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Convolution2D
# from keras.layers.convolutional import MaxPooling2D
# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder

# batch_size = 128
# epochs = 300
# num_classes = 980

# y_train = utils.to_categorical(y_train, num_classes)
# y_val = utils.to_categorical(y_val, num_classes)

# cb_early_stopping = EarlyStopping(monitor='val_loss', patience=50)

# train_generator = ImageDataGenerator(rescale=1./255, 
#                                      rotation_range=15, 
#                                      width_shift_range=0.15, 
#                                      height_shift_range=0.15, 
#                                      shear_range=0.2, 
#                                      zoom_range=[0.8, 1.2])

# val_generator = ImageDataGenerator(rescale=1./255)

# train_data_flow = train_generator.flow(X_train, y_train, 
#                                        batch_size=batch_size)
# val_data_flow = val_generator.flow(X_val, y_val,
#                                    batch_size=batch_size)

# img_rows, img_cols = 32, 32
# input_shape = (img_rows, img_cols, 3)

# model = Sequential()
# model.add(Convolution2D(32, (3, 3), padding='same',
#                  input_shape=X_train.shape[1:]))
# model.add(Activation('relu'))
# model.add(Convolution2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.50))

# model.add(Convolution2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

# RMSprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
                     
# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop,
#               metrics=['accuracy'])

# hist = model.fit_generator(train_data_flow,
#                            epochs=epochs,
#                            verbose=1,
#                            steps_per_epoch = 230,
#                            validation_data=val_data_flow,
#                            validation_steps = 70,
#                            callbacks=[cb_early_stopping])

# model.save('hand_written_korean_classification.hdf5')

In [ ]:
import cv2
import numpy as np
from skimage import io
from skimage.transform import resize
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import load_model
import pandas as pd

# 한글 데이터셋 모델 불러오기

In [ ]:
model = load_model('hand_written_korean_classification.hdf5')

In [ ]:
model.summary()

# 마우스로 자음 입력하는 코드

In [ ]:
%%writefile server.py

import flask
from tensorflow.keras.models import load_model 
from flask import Flask,render_template,url_for,request
import matplotlib.pyplot as plt
import base64
import numpy as np
import cv2

init_Base64 = 21   # data:image/png;base64, 로 시작하
app = Flask(__name__)

model = load_model('mnist_cnn.h5')

@app.route('/')
def home():
    return render_template("mnist.html")

@app.route('/upload', methods=['POST'])
def upload():        
    global model
    
    draw = request.form['url']  
    draw = draw[init_Base64:]
    draw_decoded = base64.b64decode(draw)
    image = np.asarray(bytearray(draw_decoded), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)    
    image = cv2.resize(image, dsize=(28, 28), interpolation=cv2.INTER_AREA)  
    image = image.reshape(1, 28, 28, 1)
    
    p = model.predict(image)
    
    image = image.reshape(28, 28,  1)
    print(image)
    image = cv2.resize(image, dsize=(32, 32))
    cv2.imwrite('./result.jpg', image)
    return f"result : {np.argmax(p)}"

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)
    
# mnist모델 파일 필요, 서버는 프롬프트에서 localhost:8000/ 으로 실행

# 저장 이미지 불러오기 (데이터셋이 다르기에 그림으로 불러와서 모델로 예측)

In [ ]:
newimage = cv2.imread('./result.jpg')
newimage = 255 - newimage

#검정과 흰색 변환

In [ ]:
plt.imshow(newimage)
plt.show()

In [ ]:
print(newimage.shape)

In [ ]:
test_x = []
test_x.append(newimage)

testx = np.array(test_x)
testx = testx / 255

In [ ]:
testx.shape

In [ ]:
testx

In [ ]:
print(model.predict_classes(testx))

# 라벨 번호를 출력해주기에 나온 값으로 라벨 값을 추출해내면 됨.

In [ ]:
Label = pd.read_table('./label.txt',header=None)

In [ ]:
print(Label.iloc[960])